In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from tqdm import tqdm_notebook as tqdm
import requests as req
import matplotlib.pyplot as plt

In [3]:
cafe1 = pd.read_csv("homecafe.csv", encoding = 'utf-8')
cafe1['Star'] = cafe1['Star'].str.extract(r'(\d+.\d+)')
cafe1

,Unnamed: 0.1,Unnamed: 0,Star,Date,Style,Review
0,0,0,5.0,Reviewed in the United States 🇺🇸 on 2017년 12월 15일,스타일: 머신 + 에어로색상: 그레이,"""Is this a good coffee maker?"" Well, that's go..."
1,1,1,5.0,Reviewed in the United States 🇺🇸 on 2018년 11월 28일,스타일: 기계 전용색상: 블랙,We have used this machine daily since purchase...
2,2,2,5.0,Reviewed in the United States 🇺🇸 on 2018년 12월 10일,스타일: 기계 전용색상: 그레이,The inside box that the machine came in had be...
3,3,3,5.0,Reviewed in the United States 🇺🇸 on 2022년 9월 15일,스타일: 머신 + 에어로색상: 타이탄.,I’m loving this machine! I’ve had many differe...
4,4,4,5.0,Reviewed in the United States 🇺🇸 on 2022년 9월 24일,스타일: 기계 전용색상: 블랙,This is the best purchase! My fiancé and I use...
...,...,...,...,...,...,...
2085,2085,2085,5.0,Reviewed in the United States 🇺🇸 on 2018년 1월 14일,스타일: 기계 전용색상: 그레이,SO MUCH FLAVOR.
2086,2086,2086,4.0,Reviewed in the United States 🇺🇸 on 2021년 12월 28일,스타일: 머신 + 에어로색상: 그레이,I've only had my Nespresso machine for a short...
2087,2087,2087,5.0,Reviewed in the United States 🇺🇸 on 2017년 12월 11일,스타일: 기계 전용색상: 레드,Nice and elegant
2088,2088,2088,5.0,Reviewed in the United States 🇺🇸 on 2017년 12월 15일,스타일: 기계 전용색상: 블랙,This machine has changed the way I do mornings...


In [4]:
cafe1.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Star', 'Date', 'Style', 'Review'], dtype='object')

In [5]:
cafe1.drop(['Unnamed: 0.1','Unnamed: 0'],axis=1,inplace=True)
cafe1

,Star,Date,Style,Review
0,5.0,Reviewed in the United States 🇺🇸 on 2017년 12월 15일,스타일: 머신 + 에어로색상: 그레이,"""Is this a good coffee maker?"" Well, that's go..."
1,5.0,Reviewed in the United States 🇺🇸 on 2018년 11월 28일,스타일: 기계 전용색상: 블랙,We have used this machine daily since purchase...
2,5.0,Reviewed in the United States 🇺🇸 on 2018년 12월 10일,스타일: 기계 전용색상: 그레이,The inside box that the machine came in had be...
3,5.0,Reviewed in the United States 🇺🇸 on 2022년 9월 15일,스타일: 머신 + 에어로색상: 타이탄.,I’m loving this machine! I’ve had many differe...
4,5.0,Reviewed in the United States 🇺🇸 on 2022년 9월 24일,스타일: 기계 전용색상: 블랙,This is the best purchase! My fiancé and I use...
...,...,...,...,...
2085,5.0,Reviewed in the United States 🇺🇸 on 2018년 1월 14일,스타일: 기계 전용색상: 그레이,SO MUCH FLAVOR.
2086,4.0,Reviewed in the United States 🇺🇸 on 2021년 12월 28일,스타일: 머신 + 에어로색상: 그레이,I've only had my Nespresso machine for a short...
2087,5.0,Reviewed in the United States 🇺🇸 on 2017년 12월 11일,스타일: 기계 전용색상: 레드,Nice and elegant
2088,5.0,Reviewed in the United States 🇺🇸 on 2017년 12월 15일,스타일: 기계 전용색상: 블랙,This machine has changed the way I do mornings...


In [6]:
homecafe = cafe1['Style'].str.split("색상:", expand = True)
homecafe[0] = homecafe[0].str.replace("스타일: ", "")
homecafe.rename(columns={0: "Style", 1: "Color"}, inplace=True)
homecafe

,Style,Color
0,머신 + 에어로,그레이
1,기계 전용,블랙
2,기계 전용,그레이
3,머신 + 에어로,타이탄.
4,기계 전용,블랙
...,...,...
2085,기계 전용,그레이
2086,머신 + 에어로,그레이
2087,기계 전용,레드
2088,기계 전용,블랙


In [7]:
homecafe['Star']=cafe1['Star']
homecafe['Review']=cafe1['Review']
homecafe = homecafe[['Star','Style','Color','Review']]
homecafe

,Star,Style,Color,Review
0,5.0,머신 + 에어로,그레이,"""Is this a good coffee maker?"" Well, that's go..."
1,5.0,기계 전용,블랙,We have used this machine daily since purchase...
2,5.0,기계 전용,그레이,The inside box that the machine came in had be...
3,5.0,머신 + 에어로,타이탄.,I’m loving this machine! I’ve had many differe...
4,5.0,기계 전용,블랙,This is the best purchase! My fiancé and I use...
...,...,...,...,...
2085,5.0,기계 전용,그레이,SO MUCH FLAVOR.
2086,4.0,머신 + 에어로,그레이,I've only had my Nespresso machine for a short...
2087,5.0,기계 전용,레드,Nice and elegant
2088,5.0,기계 전용,블랙,This machine has changed the way I do mornings...


In [8]:
ymd = cafe1['Date'].str.replace("Reviewed in the United States 🇺🇸 on ", "")
ymd = ymd.str.replace("년 ", "-")
ymd = ymd.str.replace("월 ", "-")
ymd = ymd.str.replace("일", "")
ymdf = pd.to_datetime(ymd)
ymdf

0      2017-12-15
1      2018-11-28
2      2018-12-10
3      2022-09-15
4      2022-09-24
          ...    
2085   2018-01-14
2086   2021-12-28
2087   2017-12-11
2088   2017-12-15
2089   2018-11-28
Name: Date, Length: 2090, dtype: datetime64[ns]

In [9]:
homecafe['Date']=ymdf
homecafe

C:\Users\user\AppData\Local\Temp\ipykernel_8252\564957419.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  homecafe['Date']=ymdf


,Star,Style,Color,Review,Date
0,5.0,머신 + 에어로,그레이,"""Is this a good coffee maker?"" Well, that's go...",2017-12-15
1,5.0,기계 전용,블랙,We have used this machine daily since purchase...,2018-11-28
2,5.0,기계 전용,그레이,The inside box that the machine came in had be...,2018-12-10
3,5.0,머신 + 에어로,타이탄.,I’m loving this machine! I’ve had many differe...,2022-09-15
4,5.0,기계 전용,블랙,This is the best purchase! My fiancé and I use...,2022-09-24
...,...,...,...,...,...
2085,5.0,기계 전용,그레이,SO MUCH FLAVOR.,2018-01-14
2086,4.0,머신 + 에어로,그레이,I've only had my Nespresso machine for a short...,2021-12-28
2087,5.0,기계 전용,레드,Nice and elegant,2017-12-11
2088,5.0,기계 전용,블랙,This machine has changed the way I do mornings...,2017-12-15


In [11]:
homecafe=homecafe.astype({'Star':float})
homecafe

,Star,Style,Color,Review,Date
0,5.0,머신 + 에어로,그레이,"""Is this a good coffee maker?"" Well, that's go...",2017-12-15
1,5.0,기계 전용,블랙,We have used this machine daily since purchase...,2018-11-28
2,5.0,기계 전용,그레이,The inside box that the machine came in had be...,2018-12-10
3,5.0,머신 + 에어로,타이탄.,I’m loving this machine! I’ve had many differe...,2022-09-15
4,5.0,기계 전용,블랙,This is the best purchase! My fiancé and I use...,2022-09-24
...,...,...,...,...,...
2085,5.0,기계 전용,그레이,SO MUCH FLAVOR.,2018-01-14
2086,4.0,머신 + 에어로,그레이,I've only had my Nespresso machine for a short...,2021-12-28
2087,5.0,기계 전용,레드,Nice and elegant,2017-12-11
2088,5.0,기계 전용,블랙,This machine has changed the way I do mornings...,2017-12-15


In [12]:
homecafe.Color.unique()

array([' 그레이', ' 블랙', ' 타이탄.', ' 레드', ' 잉크 블랙.', ' 매트 블랙.', ' LE 블랙 매트'],
      dtype=object)

In [13]:
homecafe.Style.unique()

array(['머신 + 에어로', '기계 전용', '기계 + 캡슐 VertuoLine', '머신 + 에어로치노',
       '기계 + 에어로치노 + 캡슐 세트', '기계 + 캡슐 세트'], dtype=object)

In [18]:
homecafe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2090 entries, 0 to 2089
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Star    2090 non-null   float64       
 1   Style   2090 non-null   object        
 2   Color   2090 non-null   object        
 3   Review  2083 non-null   object        
 4   Date    2090 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 81.8+ KB


In [20]:
style_1 = homecafe['Style'].value_counts()
style_1

기계 전용                 1192
머신 + 에어로               852
기계 + 캡슐 세트              18
기계 + 에어로치노 + 캡슐 세트      15
기계 + 캡슐 VertuoLine       8
머신 + 에어로치노               5
Name: Style, dtype: int64

In [189]:
cafe1['Date'].str.replace('\d+',"")

C:\Users\user\AppData\Local\Temp\ipykernel_5016\3441779624.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  cafe1['Date'].str.replace('\d+',"")


0       Reviewed in the United States 🇺🇸 on 년 월 일
1       Reviewed in the United States 🇺🇸 on 년 월 일
2       Reviewed in the United States 🇺🇸 on 년 월 일
3       Reviewed in the United States 🇺🇸 on 년 월 일
4       Reviewed in the United States 🇺🇸 on 년 월 일
                          ...                    
2085    Reviewed in the United States 🇺🇸 on 년 월 일
2086    Reviewed in the United States 🇺🇸 on 년 월 일
2087    Reviewed in the United States 🇺🇸 on 년 월 일
2088    Reviewed in the United States 🇺🇸 on 년 월 일
2089    Reviewed in the United States 🇺🇸 on 년 월 일
Name: Date, Length: 2090, dtype: object